In [1]:
import numpy as np
import pickle
import random
import json
import os
import plotly.graph_objects as go
import kaleido

from tqdm.notebook import tqdm

# A Process class which emulates a process in a server. It has a pid and a length measured in numbers of instructions.
# Furthermore, it possesses the __repr__ method which is used to print the object.
# Moreover, it possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Process:
    def __init__(self, pid, length):
        self.pid = pid
        self.length = length

    def __repr__(self):
        return f"Process(pid={self.pid}, length={self.length})"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)

In [2]:
# A Server class which emulates a server. It has a name, a number of cpus, the speed of its cpu measured in GHz and a workload list, which represents 
# the processes that are assigned to the server. The workload starts as empty. 
# Furthermore, it possesses the __repr__ method which is used to print the object.
# Moreover, it possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Server:
    def __init__(self, name, cpus, cpu_speed):
        self.name = name
        self.cpus = cpus
        self.cpu_speed = cpu_speed

    def __repr__(self):
        return f"Server(name={self.name}, cpus={self.cpus}, cpu_speed={self.cpu_speed}"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)


In [3]:
# Function to find the magnitude of a vector.
def magnitude(vector):
    return np.sqrt(np.sum(np.square(vector)))

# Function to find the module of a vector.
def module(vector):
    return np.sum(np.square(vector))

# function to rescale vector between 0 and 1 and return it as a list
def rescale_vector(vector):
    return list((vector - np.min(vector)) / (np.max(vector) - np.min(vector)))

In [4]:

# A Solution class, wich represents a solution to the problem. is made of a list of tuples in the form <server, associated processes>
# The solution object possesses a fitness value, which is a vector of the fitness of each servers with respect to their associated processes. 
# The fitness is calculated as the sum of the length of the processes assigned to the server divided by the number of cpus of the server times their speed.
# It possesses the __repr__ method which is used to print the object.
# It possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Solution:
    def __init__(self, solution):
        self.solution = solution
        self.fitness = rescale_vector(self.calculate_fitness())

    def __repr__(self):
        return f"Solution(solution={self.solution}, fitness={self.fitness})"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)

    def calculate_fitness(self):
        fitness = []
        for server, processes in self.solution:
            fitness.append(sum([process.length for process in processes]) * (server.cpus * server.cpu_speed))
        return fitness

    def crossover(self, other):
        child = []
        for i in range(len(self.solution)):
            if random.random() > 0.5:
                child.append(self.solution[i])
            else:
                child.append(other.solution[i])
        return Solution(child)
        
    def __lt__(self, other):
        if magnitude(self.fitness) < magnitude(other.fitness):
            return 1
        elif magnitude(self.fitness) > magnitude(other.fitness):
            return -1
        else:
            return 0

In [6]:
# A Particle Swarm Optimization algorithm comprised of all the necessary methods for implementing one:
# The algorithm is initialized with:
# - a list of servers coming from the path ./servers/{}.json with {} being the name of the server
# - a list of processes coming from the path ./processes/{}.json with {} being the name of the process
# - a number of particles
# - a number of iterations

class PSO:
    def __init__(self, servers, processes, particles, iterations):
        self.servers = servers
        self.processes = processes
        self.particles = particles
        self.iterations = iterations
        self.swarm = []
        self.gbest = None
        self.pbest = None
        self.w = 0.9
        self.c1 = 0.5
        self.c2 = 0.5
        self.vmax = 1

    def __repr__(self):
        return f"PSO(servers={self.servers}, processes={self.processes}, particles={self.particles}, iterations={self.iterations})"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)

    # Function to initialize the swarm with random solutions
    def initialize_swarm(self):
        self.swarm = []
        for i in range(self.particles):
            solution = []
            for server in self.servers:
                solution.append((server, []))
            for process in self.processes:
                solution[random.randint(0, len(solution) - 1)][1].append(process)
            self.swarm.append(Solution(solution))

    # Function to initialize the pbest and gbest
    def initialize_pbest_gbest(self):
        self.pbest = self.swarm
        self.gbest = self.pbest[0]

    # Function to update the pbest and gbest
    def update_pbest_gbest(self):
        for i in range(len(self.pbest)):
            if self.pbest[i] < self.swarm[i]:
                self.pbest[i] = self.swarm[i]
            if self.pbest[i] < self.gbest:
                self.gbest = self.pbest[i]

    # Function to update the velocity of each particle
    def update_velocity(self):
        for i in range(len(self.swarm)):
            for j in range(len(self.swarm[i].solution)):
                for k in range(len(self.swarm[i].solution[j][1])):
                    self.swarm[i].solution[j][1][k].length += random.uniform(-self.vmax, self.vmax)

    # Function to update the position of each particle
    def update_position(self):
        for i in range(len(self.swarm)):
            for j in range(len(self.swarm[i].solution)):
                for k in range(len(self.swarm[i].solution[j][1])):
                    self.swarm[i].solution[j][1][k].length = max(0, self.swarm[i].solution[j][1][k].length)

    # Function to update the swarm
    def update_swarm(self):
        self.update_velocity()
        self.update_position()
        self.update_pbest_gbest()

    # Function to run the algorithm
    def run(self):
        self.initialize_swarm()
        self.initialize_pbest_gbest()
        for i in tqdm(range(self.iterations)):
            self.update_swarm()
        return self.gbest

In [17]:
# An improvement of the PSO algorithm, called PSOPlus, which implements the strategy called "local search" to improve the results of the PSO algorithm.
# The algorithm is initialized with:
# - a list of servers coming from the path ./servers/{}.json with {} being the name of the server
# - a list of processes coming from the path ./processes/{}.json with {} being the name of the process
# - a number of particles
# - a number of iterations
# - a number of local search iterations

class PSOPlus(PSO):
    def __init__(self, servers, processes, particles, iterations, local_search_iterations):
        super().__init__(servers, processes, particles, iterations)
        self.local_search_iterations = local_search_iterations

    def __repr__(self):
        return f"PSOPlus(servers={self.servers}, processes={self.processes}, particles={self.particles}, iterations={self.iterations}, local_search_iterations={self.local_search_iterations})"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)

    # Function to run the algorithm
    def run(self):
        self.initialize_swarm()
        self.initialize_pbest_gbest()
        for i in tqdm(range(self.iterations)):
            self.update_swarm()
            for j in range(self.local_search_iterations):
                for k in range(len(self.swarm)):
                    self.swarm[k] = self.swarm[k].crossover(self.gbest)
        return self.gbest

In [18]:
with open(f"./servers/0.pickle", "rb") as f:
    servers = pickle.load(f)
with open(f"./processes/0.pickle", "rb") as f:
    processes = pickle.load(f)

In [19]:
particles = 100
iterations = 1000
pso = PSO(servers, processes, particles, iterations)
psoplus = PSOPlus(servers, processes, particles, iterations, 100)

In [21]:
gbest = pso.run()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [22]:
gbestplus = psoplus.run()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [26]:
a = []
for p in pso.pbest:
    a.append(magnitude(p.fitness))
min(a)

1.3002539775462334

In [27]:
b = []
for p in psoplus.pbest:
    b.append(magnitude(p.fitness))
min(b)

1.3816927281336238